# This module shows how my GGNN can be run (train\test loop etc.) and shows some initial results, which outperform the baselines in Baselines.ipynb

In [1]:
import csv
import torch as tt
from torch.utils.data import DataLoader
import time
import os
import json
import numpy as np
import random
from tqdm.notebook import tqdm

import sys
sys.path.insert(0, "../python/")
from utils import CustomTripletLoss
from data_loader import GraphDataset
from model import GGNN

seed = 0
random.seed(seed)
np.random.seed(seed)
tt.manual_seed(seed)

In [2]:
CUDA = True

## Preparing the datasets:

In [3]:
def load_dataset(directory, filename, batch_size, shuffle, targets, 
                 hidden_size=20, max_nodes=300, edge_types=3, annotation_size=20, target_edge_type=1):
    """
    Load a .json file into memory, create a GraphDataset out of it and return a DataLoader for it
    :param directory:       the directory from which to load the file
    :param filename:        the name of the file
    :param batch_size:      batch size used to initialize the DataLoader
    :param shuffle:         if True, shuffle the graphs on each pass
    :param targets:         Can be either "generate", "generateOnPass", or a key to the json dictionary
                            from which to load the targets
                            "generate": generate targets once and keep them this way (validation)
                            "generateOnPass": generate new targets at each epoch (training)
    :param hidden_size:     the size of node embedding in GGNN that will be used on this dataset
    :param max_nodes:       maximum number of nodes per graph
    :param edge_types:      number of different edge-types. Does not include the edges added to
                            the undirected graph
    :param annotation_size: the size of annotations (initial embedddings) for each node
    :param target_edge_type:the type of edge that is to be predicted
    :return:                a dataloader object
    """
    full_path = os.path.join(directory, filename) # full path to the file
    print("Loading data from %s" % full_path)
    with open(full_path, 'r') as f:
        data = json.load(f)
    dataset =  GraphDataset(data,
                            hidden_size=hidden_size,
                            max_nodes=max_nodes,
                            edge_types=edge_types,
                            annotation_size=annotation_size,
                            targets=targets,
                            target_edge_type=target_edge_type)
    return DataLoader(dataset, shuffle=shuffle, batch_size=batch_size, num_workers=4)

In [4]:
DIR = '../data/graphs/newMethod/'
test_data = load_dataset(DIR, 'test.json', batch_size=10, shuffle=False, targets="targets_1")
train_data = load_dataset(DIR, "train.json", batch_size=10, shuffle=True, targets="generateOnPass")
val_data = load_dataset(DIR, 'valid.json', batch_size=10, shuffle=False, targets="generate")

Loading data from ../data/graphs/newMethod/test.json
Loading data from ../data/graphs/newMethod/train.json
Loading data from ../data/graphs/newMethod/valid.json


## Epoch\training loops

In [5]:
def run_epoch(model, data, epoch, is_training):
    """
    Run a given model for one epoch on the given dataset and return the loss and the accuracy
    :param data:         a DataLoader object that works on a GraphDataset
    :param epoch:        epoch id (for logging purposes)
    :param is_training:  whether to train or just evaluate
    :return:             mean loss, mean accuracy
    """
    if CUDA:
        model.cuda()

    if is_training:
        model.train()
        print("Epoch %d. Training" % epoch)
    else:
        model.eval()
        print("Epoch %d. Evaluating" % epoch)

    total_loss = 0
    total_acc = 0
    step = 0
    batches = tqdm(data)

    for adj_matrix, features, src, mask in batches:
        step += 1

        optimizer.zero_grad()
        model.zero_grad()

        batch_size = adj_matrix.shape[0]
        option_size = adj_matrix.shape[1]

        # TODO: move these view functions in the GraphDataset
        adj_matrix = adj_matrix.view(-1, adj_matrix.shape[2], adj_matrix.shape[3]).float()
        src = src.view(-1).long()
        features = features.view(-1, features.shape[2], features.shape[3]).float()

        if CUDA:  # move to CUDA, if possible
            mask = mask.cuda()
            features = features.cuda()
            adj_matrix = adj_matrix.cuda()
            src = src.cuda()

        distances = model(features, adj_matrix, src, batch_size, option_size)

        loss, acc = criterion(distances, mask)
        total_acc += acc.cpu().data.numpy().item()
        total_loss += loss.cpu().data.numpy().item()

        batches.set_description("Acc: step=%.2f, mean=%.2f. Mean loss=%.2f" % (acc, total_acc / step, total_loss / step))

        if is_training:
            loss.backward(retain_graph=True)
            tt.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

    return total_loss/step, total_acc/step

In [6]:
def train(model, epochs, patience, train_data, val_data, best_model_file):
    """
    Train a given model for a given number of epochs. Use early stopping with given patience.
    :param model:     a GGNN model
    :param epochs:    maximum number of epochs to run the model for
    :param patience:  patience value to use for early stopping
    :param train_data:training dataset
    :apram val_data:  validation dataset
    :param best_model_file: file to save the best model to
    """

    best_val_loss, best_val_loss_epoch = float("inf"), 0
    for epoch in range(epochs):
        run_epoch(model, train_data, epoch, True)
        val_loss, _ = run_epoch(model, val_data, epoch, False)

        if val_loss < best_val_loss:
            tt.save(model.state_dict(), best_model_file)
            print("Best epoch so far. Saving to '%s')" % (best_model_file))
            best_val_loss = val_loss
            best_val_loss_epoch = epoch
        elif epoch - best_val_loss_epoch >= patience:  # early stopping
            print("Early Stopping after %i epochs." % patience)
            break

## Creating, training, and evaluating the model

In [7]:
model = GGNN(state_dim = 20,
             annotation_dim = 20,
             n_edge_types=3,
             n_nodes=300,
             n_steps=6)
criterion = CustomTripletLoss(margin=0.5)
optimizer = tt.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
train(model, epochs=20, patience=3, train_data=train_data, val_data=val_data, best_model_file = "best.model")

Epoch 0. Training



Epoch 0. Evaluating



Best epoch so far. Saving to 'best.model')
Epoch 1. Training



Epoch 1. Evaluating


--- Logging error ---
--- Logging error ---
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/af9562/.conda/envs/import/lib/python3.8/logging/__init__.py", line 1085, in emit
    self.flush()
--- Logging error ---
  File "/home/af9562/.conda/envs/import/lib/python3.8/logging/__init__.py", line 1085, in emit
    self.flush()
  File "/home/af9562/.conda/envs/import/lib/python3.8/logging/__init__.py", line 1065, in flush
    self.stream.flush()
OSError: [Errno 5] Input/output error
Traceback (most recent call last):
  File "/home/af9562/.conda/envs/import/lib/python3.8/logging/__init__.py", line 1065, in flush
    self.stream.flush()
OSError: [Errno 5] Input/output error
Call stack:
  File "/home/af9562/.conda/envs/import/lib/python3.8/logging/__init__.py", line 1085, in emit
    self.flush()
Call stack:
  File "/home/af9562/.conda/envs/import/lib/python3.8/logging/__init__.py", line 1065, in flush
    self.stream.flush()
OSError: [Errno 5] Input/output e

...

  File "/home/af9562/.conda/envs/import/lib/python3.8/site-packages/ipywidgets/widgets/widget.py", line 489, in send_state
    self._send(msg, buffers=buffers)
  File "/home/af9562/.conda/envs/import/lib/python3.8/site-packages/ipywidgets/widgets/widget.py", line 737, in _send
    self.comm.send(data=msg, buffers=buffers)
  File "/home/af9562/.conda/envs/import/lib/python3.8/site-packages/ipykernel/comm/comm.py", line 122, in send
    self._publish_msg('comm_msg',
  File "/home/af9562/.conda/envs/import/lib/python3.8/site-packages/ipykernel/comm/comm.py", line 66, in _publish_msg
    self.kernel.session.send(self.kernel.iopub_socket, msg_type,
Message: 'WARNING: attempted to send message from fork\n%s'
Arguments: {'header': {'msg_id': 'de5896a1-86e5805ce9e294b9413f47bb_45671', 'msg_type': 'comm_msg', 'username': 'af9562', 'session': 'de5896a1-86e5805ce9e294b9413f47bb', 'date': datetime.datetime(2020, 4, 5, 18, 8, 52, 846779, tzinfo=datetime.timezone.utc), 'version': '5.3'}, 'msg_id': '


Epoch 7. Evaluating


--- Logging error ---
Traceback (most recent call last):
  File "/home/af9562/.conda/envs/import/lib/python3.8/logging/__init__.py", line 1085, in emit
    self.flush()
  File "/home/af9562/.conda/envs/import/lib/python3.8/logging/__init__.py", line 1065, in flush
    self.stream.flush()
OSError: [Errno 5] Input/output error
Call stack:
  File "/home/af9562/.conda/envs/import/lib/python3.8/threading.py", line 890, in _bootstrap
    self._bootstrap_inner()
  File "/home/af9562/.conda/envs/import/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/af9562/.conda/envs/import/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/af9562/.conda/envs/import/lib/python3.8/multiprocessing/resource_sharer.py", line 139, in _serve
    signal.pthread_sigmask(signal.SIG_BLOCK, signal.valid_signals())
  File "/home/af9562/.conda/envs/import/lib/python3.8/signal.py", line 82, in valid_signals
    return {_int_to_en

  File "/home/af9562/.conda/envs/import/lib/python3.8/site-packages/traitlets/traitlets.py", line 1134, in _notify_trait
    self.notify_change(Bunch(
  File "/home/af9562/.conda/envs/import/lib/python3.8/site-packages/ipywidgets/widgets/widget.py", line 605, in notify_change
    self.send_state(key=name)
  File "/home/af9562/.conda/envs/import/lib/python3.8/enum.py", line 567, in __new__
    return cls._value2member_map_[value]
  File "/home/af9562/.conda/envs/import/lib/python3.8/site-packages/tqdm/std.py", line 1066, in __del__
    self.close()
  File "/home/af9562/.conda/envs/import/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/af9562/.conda/envs/import/lib/python3.8/site-packages/tqdm/notebook.py", line 243, in close
    self.sp(bar_style='success')
  File "/home/af9562/.conda/envs/import/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/af9562/.conda/envs/import/lib/python3.8/site-pa


Early Stopping after 3 epochs.


In [9]:
model.load_state_dict(tt.load("best.model"))

<All keys matched successfully>

In [10]:
loss, acc = run_epoch(model, test_data, 1, False)
print(acc)

Epoch 1. Evaluating



0.8162607100496094
